In [4]:
import nibabel as nib
import numpy as np
import matplotlib.colors as mcolors
from nilearn import plotting
from nilearn import datasets
import matplotlib.cm as mplcm
import os
import ipywidgets as widgets
from ipywidgets import interact
from plotly.subplots import make_subplots

In [ ]:
@interact(surface=widgets.ToggleButtons(options=('inflated' ,'white', 'pial'), description=('Cortical Surface')),
          hemi_name=widgets.ToggleButtons(options=('left', 'right'), description=('Hemisphere')),
          measure=widgets.ToggleButtons(options=('volume', 'thickness', 'area'), value='thickness', description=('Cortical metric')),
          contrast=widgets.ToggleButtons(options=('all', 'Prg', 'Post', 'Overlap'), description=('Contrast')),
          alpha=widgets.FloatSlider(value=0.05, min=0.0, max=0.2, step=0.001, description='alpha', readout_format='.3f'))
def plot_cross_differences(surface, hemi_name, measure, contrast, alpha):
    fsaverage = datasets.fetch_surf_fsaverage('fsaverage') 
    results_dir = os.path.join(os.getcwd(), 'main_data', 'without_covariates')
    hemi = '{}h'.format(hemi_name[0])
    if surface == 'inflated':
        surface = 'infl'
    surf = fsaverage[f'{surface}_{hemi_name}']
    surf_bg = fsaverage[f'curv_{hemi_name}']

    stat = 'FDRcorrP_across_hemi'
    one_minus_alpha = 1.0 - alpha

    # Defining colormaps and threshold for the plots, and custom handles for the legend
    col = ['#FFFDA2', '#FFFDA2','#FFAB76','#98A8F8']
    our_cmap = mcolors.LinearSegmentedColormap.from_list("", col)
    thr_val = 1.0

    # Plotting
    # Loading the fdr-pvalue maps 
    fdrcorrp_prg_map_name = f"{hemi}.{measure}_sm10_{stat}_contrast_sesPrg.mgh"
    fdrcorrp_post_map_name = f"{hemi}.{measure}_sm10_{stat}_contrast_sesPost.mgh"
    ses_prg_mfdrp_data = nib.load(f"{results_dir}/{fdrcorrp_prg_map_name}").get_fdata()
    ses_post_mfdrp_data = nib.load(f"{results_dir}/{fdrcorrp_post_map_name}").get_fdata()
    # Loading the sign maps 
    sgn_prg_map_name = f"{hemi}.{measure}_sm10_sgn_contrast_sesPrg.mgh"
    sgn_post_map_name = f"{hemi}.{measure}_sm10_sgn_contrast_sesPost.mgh"
    ses_prg_sgn_data = nib.load(f"{results_dir}/{sgn_prg_map_name}").get_fdata()
    ses_post_sgn_data = nib.load(f"{results_dir}/{sgn_post_map_name}").get_fdata()
    # Masking voxels with Controls > Mothers
    ses_prg_mfdrp_data[ses_prg_sgn_data > 0] = 0
    ses_post_mfdrp_data[ses_post_sgn_data > 0] = 0
    # Computing thresholded significance maps
    ses_prg_sig_map = np.where(ses_prg_mfdrp_data < one_minus_alpha, 0, 1)
    ses_post_sig_map = np.where(ses_post_mfdrp_data < one_minus_alpha, 0, 1)
    # Merge Pre and Post thresholded significance maps in one (1: Prg, 2: Post, 3: Overlap)
    if contrast=='all':
        surf_map = ses_prg_sig_map + ses_post_sig_map*2
    if contrast == 'Prg':
        surf_map = ses_prg_sig_map
    if contrast == 'Post':
        surf_map = ses_post_sig_map*2
    if contrast == 'Overlap':
        surf_map = 3.0*(np.logical_and(ses_post_sig_map, ses_prg_sig_map))
    plot = plotting.plot_surf_roi(surf,
                           surf_map,
                           hemi=hemi_name,
                           vmax=3,
                           threshold=thr_val,
                           bg_map=surf_bg,
                           cmap=our_cmap,
                           engine='plotly')
    plot.show(renderer='notebook')

In [ ]:
@interact(model=widgets.ToggleButtons(options=(['without_covariates', 'covariates_euler_age_etiv', 'covariates_euler_age_etiv_psqi_pss', 'labor_analysis']), description=('Model')),
          surface=widgets.ToggleButtons(options=('inflated' ,'white', 'pial'), description=('Cortical Surface')),
          hemi_name=widgets.ToggleButtons(options=('left', 'right'), description=('Hemisphere')),
          measure=widgets.ToggleButtons(options=('volume', 'thickness', 'area'), value='thickness', description=('Cortical metric')),
          contrast=widgets.ToggleButtons(options=('Prg', 'Post', 'Prg-to-Post'), description=('Contrast')),
          alpha=widgets.FloatSlider(value=0.05, min=0.0, max=0.2, step=0.001, description='alpha', readout_format='.3f'))
def plot_signed_effect_sizes(model, surface, hemi_name, measure, contrast, alpha):
    fsaverage = datasets.fetch_surf_fsaverage('fsaverage') 
    if model == 'labor_analysis':
        results_dir = os.path.join('.', 'main_data', model, 'without_covariates')
    else:
        results_dir = os.path.join('.', 'main_data', model)
    hemi = '{}h'.format(hemi_name[0])
    if surface == 'inflated':
        surface = 'infl'
    surf = fsaverage[f'{surface}_{hemi_name}']
    surf_bg = fsaverage[f'curv_{hemi_name}']
    stat = 'FDRcorrP_across_hemi'
    eff_size = 'partialETA2'
    one_minus_alpha = 1.0 - alpha
    vmax_value = 0.14
    threshold = 0.001
    colors_pos = mplcm.autumn(np.linspace(0, 1, 128))
    colors_neg = mplcm.winter(np.linspace(1, 0, 128))
    colors = np.vstack((colors_neg, colors_pos))
    our_cmap = mcolors.LinearSegmentedColormap.from_list('my_colormap', colors)
    if contrast == 'Prg-to-Post':
        contrast = 'groupxses'
    else:
        contrast = f'ses{contrast}'
    mfdrp_map_name = f"{hemi}.{measure}_sm10_{stat}_contrast_{contrast}.mgh"
    eff_size_map_name = f"{hemi}.{measure}_sm10_{eff_size}_contrast_{contrast}.mgh"
    sgn_map_name = f"{hemi}.{measure}_sm10_sgn_contrast_{contrast}.mgh"
    long_mfdrp_data = nib.load(f"{results_dir}/{mfdrp_map_name}").get_fdata()
    long_eff_size_data = nib.load(f"{results_dir}/{eff_size_map_name}").get_fdata()
    long_sgn_data = nib.load(f"{results_dir}/{sgn_map_name}").get_fdata()
    # Computing thresholded signed effect size maps
    surf_map = long_sgn_data * long_eff_size_data * (long_mfdrp_data > one_minus_alpha)
    plot=plotting.plot_surf_stat_map(surf,
                                       surf_map,
                                       hemi=hemi_name,
                                       threshold=threshold,
                                       vmax=vmax_value,
                                       bg_map=surf_bg,
                                       cmap=our_cmap,
                                       colorbar=True,
                                       engine='plotly')
    plot.figure.data[1].colorbar.tickformat='.2f'
    plot.show(renderer='notebook')

interactive(children=(ToggleButtons(description='Model', options=('without_covariates', 'covariates_euler_age_…